In [ ]:
# /###########################
# Imports
###########################  

import gurobipy as gp
from gurobipy import GRB 

import matplotlib.pyplot as plt

from datetime import date
import math
import networkx as nx
import csv
import time
import json
import sys
import os

import hess
import labeling
import ordering
import fixing
import separation

from gerrychain import Graph
import geopandas as gpd


################################################
# Summarize computational results to csv file
################################################ 

from csv import DictWriter
def append_dict_as_row(file_name, dict_of_elem, field_names):
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        dict_writer = DictWriter(write_obj, fieldnames=field_names)
        #dict_writer = DictWriter(write_obj, )
        # Add dictionary as wor in the csv
        dict_writer.writerow(dict_of_elem)
        
        
################################################
# Writes districting solution to json file
################################################ 

def export_to_json(G, districts, filename):
    with open(filename, 'w') as outfile:
        soln = {}
        soln['nodes'] = []
        for j in range(len(districts)):
            for i in districts[j]:
                soln['nodes'].append({
                        'name': G.nodes[i]["NAME20"],
                        'index': i,
                        'GEOID20': G.nodes[i]['GEOID20'],
                        'GEOID': G.nodes[i]['GEOID'],
                        'district': j
                        })
        json.dump(soln, outfile, indent=4)

               

################################################
# Draws districts and saves to png file
################################################ 

def export_to_png(G, df, districts, filename):
    
    assignment = [ -1 for u in G.nodes ]
    
    for j in range(len(districts)):
        for i in districts[j]:
            geoID = G.nodes[i]["GEOID20"]
            for u in G.nodes:
                if geoID == df['GEOID20'][u]:
                    assignment[u] = j
    
    if min(assignment[v] for v in G.nodes) < 0:
        print("Error: did not assign all nodes in district map png.")
    else:
        df['assignment'] = assignment
        my_fig = df.plot(column='assignment').get_figure()
        RESIZE_FACTOR = 3
        my_fig.set_size_inches(my_fig.get_size_inches()*RESIZE_FACTOR)
        plt.axis('off')
        my_fig.savefig(filename)
        
        
        
        
        dissolved = df.dissolve(by = 'assignment',aggfunc={'P0030004': 'sum', 'P0030001': 'sum'})
        
        plot_bvap(df,filename,'heatmap', overlay = True, gdf2 = dissolved)
        plot_bvap(dissolved, filename)
        

 
        
################################################
# Draws max B set and saves to png file
################################################ 

def export_B_to_png(G, df, B, filename):
    
    B_geoids = [ G.nodes[i]["GEOID20"] for i in B ]
    df['B'] = [1 if df['GEOID20'][u] in B_geoids else 0 for u in G.nodes]
        
    my_fig = df.plot(column='B').get_figure()
    RESIZE_FACTOR = 3
    my_fig.set_size_inches(my_fig.get_size_inches()*RESIZE_FACTOR)
    plt.axis('off')
    my_fig.savefig(filename)

    
def plot_bvap(dissolved,filename, additional_file_name = '', overlay = False, gdf2 = 0):
        dissolved['Representative'] = dissolved['geometry'].apply(lambda x: x.representative_point().coords[:])
        dissolved['Representative'] = [coords[0] for coords in dissolved['Representative']]
        dissolved = dissolved.rename(columns={'P0010001':'POP','P0010004':'BPOP',
                                                  'P0030001':'VAP','P0030004':'BVAP', 'P0040002' : 'HVAP'})
        dissolved['BVAP_ratio'] = dissolved['BVAP']/dissolved['VAP']
        colormap ='BuPu'
        
        fig, ax = plt.subplots(1,1, figsize=(16,16))
        
        cbax = fig.add_axes([0.95, 0.3, 0.03, 0.39])   
        cbax.set_title("BVAP Ratio")
        vmin = 0
        vmax = 0.75
        sm = plt.cm.ScalarMappable(cmap=colormap, \
                norm=plt.Normalize(vmin=vmin, vmax=vmax))
    
        sm._A = []
    
        fig.colorbar(sm, cax=cbax)#, format="%d")
    
        ax.axis("off")

        dissolved.plot(column = 'BVAP_ratio', edgecolor='black', cmap=colormap, ax = ax, vmin = vmin, vmax = vmax, legend=False)
        #dissolved.apply(lambda x: 
             #ax.annotate(text=x.NAME, xy=x.Representative, ha='center', color = 'r',  weight='bold', size = 14), axis=1);
        
        #dissolved.set_index('my_index', inplace=True)
        dissolved.reset_index(inplace=True)
        #dissolved.apply(lambda x: 
             #ax.annotate(text=str(x.name) +': '+ str(x.BVAP), xy=x.Representative, ha='center', color = 'r',  weight='bold', size = 14), axis=1);
        
        dissolved.apply(lambda x: 
             ax.annotate(text=str(x.name) +': '+ str(round(labeling.cdf_fun(x.BVAP_ratio),2)), xy=x.Representative, ha='center', color = 'r',  weight='bold', size = 14), axis=1);
        ratios = list(dissolved['BVAP_ratio'])
        
        score = sum(labeling.cdf_fun(r) for r in ratios)
        ax.set_title(f"BVAP Score: {score}")
        if overlay:
            gdf2.plot(ax=ax, edgecolor='blue', linewidth=2, facecolor='none')
        plt.savefig(filename.split(".")[0] + '_BVAP' + additional_file_name + '.png', transparent=True)

###########################
# Hard-coded inputs
###########################  

state_codes = {
    'WA': '53', 'DE': '10', 'WI': '55', 'WV': '54', 'HI': '15',
    'FL': '12', 'WY': '56', 'NJ': '34', 'NM': '35', 'TX': '48',
    'LA': '22', 'NC': '37', 'ND': '38', 'NE': '31', 'TN': '47', 'NY': '36',
    'PA': '42', 'AK': '02', 'NV': '32', 'NH': '33', 'VA': '51', 'CO': '08',
    'CA': '06', 'AL': '01', 'AR': '05', 'VT': '50', 'IL': '17', 'GA': '13',
    'IN': '18', 'IA': '19', 'MA': '25', 'AZ': '04', 'ID': '16', 'CT': '09',
    'ME': '23', 'MD': '24', 'OK': '40', 'OH': '39', 'UT': '49', 'MO': '29',
    'MN': '27', 'MI': '26', 'RI': '44', 'KS': '20', 'MT': '30', 'MS': '28',
    'SC': '45', 'KY': '21', 'OR': '41', 'SD': '46'
}

number_of_congressional_districts = {
    'WA': 10, 'DE': 1, 'WI': 8, 'WV': 3, 'HI': 2,
    'FL': 27, 'WY': 1, 'NJ': 12, 'NM': 3, 'TX': 36,
    'LA': 6, 'NC': 13, 'ND': 1, 'NE': 3, 'TN': 9, 'NY': 27,
    'PA': 18, 'AK': 1, 'NV': 4, 'NH': 2, 'VA': 11, 'CO': 7,
    'CA': 53, 'AL': 7, 'AR': 4, 'VT': 1, 'IL': 18, 'GA': 14,
    'IN': 9, 'IA': 4, 'MA': 9, 'AZ': 9, 'ID': 2, 'CT': 5,
    'ME': 2, 'MD': 8, 'OK': 5, 'OH': 16, 'UT': 4, 'MO': 8,
    'MN': 8, 'MI': 14, 'RI': 2, 'KS': 4, 'MT': 1, 'MS': 4,
    'SC': 7, 'KY': 6, 'OR': 5, 'SD': 1
}

default_config = {
    'state' : 'OK',
    'level' : 'county',
    'base' : 'labeling',
    'obj' : 'step',
    'R' : '10',
    'tlimit' : 3600,
    'dist_bounds' : False,
    'fixing' : True,
    'contiguity' : 'scf',
    'symmetry' : 'orbitope',
    'extended' : False,
    'order' : 'B_decreasing',
    'heuristic' : True,
    'lp': True
}

available_config = {
    'state' : { key for key in state_codes.keys() },
    'level' : {'county', 'tract'},
    'base' : {'hess', 'labeling'},
    'obj' : {"LogEPWL", "BNPWL", "perimin", "perimax", "compact", "step-hvap", "step-alt", "DiscretePWL", "PWL", "step-exp", "step-max", "cumulative", "conv", "PWL_approx", "conv_approx", "partisan_dem","partisan_dem_order", "partisan_rep","competetive","step-ordered"},
    'R' : int,
    'fixing' : {True, False},
    'contiguity' : {'none', 'lcut', 'scf', 'shir'},
    'symmetry' : {'default', 'aggressive', 'orbitope'},  # orbitope only for labeling
    'extended' : {True, False},
    'order' : {'none', 'decreasing', 'B_decreasing'},
    'heuristic' : {True, False},
    'lp' : {True, False}, # solve and report root LP bound? (in addition to MIP)
    'obj_order' : {'D20_max', 'D16_max','T20_max','T16_max','D20_min','D16_min','T20_min','T16_min'},
    'index':{0,1,2,3,4,5,6,7,8,9,10,11,12,13,14}
}


###############################################
# Read configs/inputs and set parameters
############################################### 

# read configs file and load into a Python dictionary

# if len(sys.argv)>1:
#     # name your own config file in command line, like this: 
#     #       python main.py usethisconfig.json
#     # to keep logs of the experiments, redirect to file, like this:
#     #       python main.py usethisconfig.json 1>>log_file.txt 2>>error_file.txt
#     config_filename = sys.argv[1] 
# else:

config_filename = 'config-rob-tests.json' # default
config_filename = 'config-rob-tests-BNPWL.json' # default
#config_filename = 'config-rob-tests-DEM.json' # default
#config_filename = 'config-political-bounds_5min.json'
    
print("Reading config from",config_filename)    
config_filename_wo_extension = config_filename.rsplit('.',1)[0]
configs_file = open(config_filename,'r')
batch_configs = json.load(configs_file)
configs_file.close()

# create directory for results
path = os.path.join("..", "results_for_" + config_filename_wo_extension) 
if not os.path.exists(path):
    os.mkdir(path) 

# print results to csv file
today = date.today()
today_string = today.strftime("%Y_%b_%d") # Year_Month_Day, like 2019_Sept_16
results_filename = "../results_for_" + config_filename_wo_extension + "/results_" + config_filename_wo_extension + "_" + today_string + ".csv" 

# prepare csv file by writing column headers
with open(results_filename,'w',newline='') as csvfile:   
    my_fieldnames = ['run','state','level','base','obj','R','tlimit','fixing','contiguity','symmetry','extended','order','heuristic','lp'] # configs
    my_fieldnames += ['k','L','U','n','m'] # params
    my_fieldnames += ['heur_obj', 'heur_time', 'heur_iter'] # heuristic info
    my_fieldnames += ['B_q', 'B_size', 'B_time', 'B_timelimit'] # max B info
    my_fieldnames += ['DFixings', 'LFixings', 'UFixings_X', 'UFixings_R', 'ZFixings'] # fixing info
    my_fieldnames += ['LP_obj', 'LP_time'] # root LP info
    my_fieldnames += ['MIP_obj','MIP_bound','MIP_time', 'MIP_timelimit', 'MIP_status', 'MIP_nodes', 'callbacks', 'lazy_cuts', 'connected'] # MIP info
    my_fieldnames += ['dist_bounds']
    my_fieldnames += ['expected_error','maximum_error']
    writer = csv.DictWriter(csvfile, fieldnames = my_fieldnames)
    writer.writeheader()

    
############################################################
# Run experiments for each config in batch_config file
############################################################

for key in batch_configs.keys(): 
      
    # get config and check for errors
    config = batch_configs[key]
    print("In run",key,"using config:",config,end='.')
    for ckey in available_config.keys():
        if not ckey =='R':
            if config[ckey] not in available_config[ckey]:
                errormessage = "Error: the config option"+ckey+":"+config[ckey]+"is not known."
                sys.exit(errormessage)
    print("")
    
    # fill-in unspecified configs using default values
    for ckey in available_config.keys():
        if ckey not in config.keys():
            print("Using default value",ckey,"=",default_config[ckey],"since no option was selected.")
            config[ckey] = default_config[ckey]
        
    # initialize dictionary to store this run's results
    result = config
    result['run'] = key            
                   
    # read input data
    state = config['state']
    code = state_codes[state]
    level = config['level']
    print("../data/"+level+"/dual_graphs/"+level+code+".json")
    G = Graph.from_json("../data/"+level+"/dual_graphs/"+level+code+".json")
    print(G)
    
    DG = nx.DiGraph(G) # bidirected version of G
    df = gpd.read_file("../data/"+level+"/shape_files/"+state+"_"+level+".shp")

    # set parameters
    k = number_of_congressional_districts[state]        
    population = {node : G.nodes[node]['POP100'] for node in G.nodes()}  
    deviation = .2
    L = math.ceil((1-deviation/2)*sum(population[node] for node in G.nodes())/k)
    U = math.floor((1+deviation/2)*sum(population[node] for node in G.nodes())/k)
    print("L =",L,", U =",U,", k =",k)
    result['k'] = k
    result['L'] = L
    result['U'] = U
    result['m'] = G.number_of_edges()
    result['n'] = G.number_of_nodes()
    
    # abort early for trivial or overtly infeasible instances
    maxp = max(population[i] for i in G.nodes)
    if k==1 or maxp>U:
        print("k=",k,", max{ p_v | v in V } =",maxp,", U =",U,end='.')
        sys.exit("Aborting early, either due to trivial instance or overtly infeasible instance.")
           
    # read heuristic solution from external file (?)
    heuristic = config['heuristic']
    result['heur_obj'] = 'n/a'
    result['heur_time'] = 'n/a'
    result['heur_iter'] = 'n/a'
        
          
    ############################
    # Build base model
    ############################   
    
    m = gp.Model()
    m._DG = DG
    base = config['base']
    
    if base == 'hess':
        # X[i,j]=1 if vertex i is assigned to (district centered at) vertex j
        m._X = m.addVars(DG.nodes, DG.nodes, vtype=GRB.BINARY)
        hess.add_base_constraints(m, population, L, U, k)
    
    if base == 'labeling':        
        # X[i,j]=1 if vertex i is assigned to district j in {0,1,2,...,k-1}
        m._X = m.addVars(DG.nodes, range(k), vtype=GRB.BINARY, name="X")
        if config['symmetry']=='orbitope' or config['contiguity'] in {'scf', 'shir', 'lcut'}:
            m._R = m.addVars(DG.nodes, range(k), vtype=GRB.BINARY)
        labeling.add_base_constraints(m, population, L, U, k)

                
    ############################################
    # Add (extended?) objective 
    ############################################
    
    extended = config['extended']
    obj = config['obj']
    
    if base == 'hess':
        if extended:
            hess.add_extended_objective(m, G)
        else:
            hess.add_objective(m, G)
    print(config)
    
    R=config['R']
    
    dist_bounds = config["dist_bounds"]
    if base == 'labeling':
        # if extended:
        #     labeling.add_extended_objective(m, G, k)`
        # else:
        #     labeling.add_objective(m, G, k)
        if obj == "PWL":
            labeling.add_PWL_objective(m, G, k, R)
            if heuristic:
                heuristic_districts = [ [node  for node in G.nodes  if G.nodes[node]['start_bvap']==j+1] for j in range(k) ]
            else:
                heuristic_districts = None
        elif obj == "step-exp":
            (expErr,maxErr) = labeling.add_step_exp_objective(m, G, k, R)
            if heuristic:
                heuristic_districts = [ [node  for node in G.nodes  if G.nodes[node]['start_bvap']==j+1] for j in range(k) ]
            else:
                heuristic_districts = None
        elif obj == "step-max":
            (expErr,maxErr) = labeling.add_step_max_objective(m, G, k, R)
            if heuristic:
                heuristic_districts = [ [node  for node in G.nodes  if G.nodes[node]['start_bvap']==j+1] for j in range(k) ]
            else:
                heuristic_districts = None
        elif obj == "cumulative":
            labeling.add_cumulative_objective(m, G, k)
        elif obj == "conv":
            labeling.add_conv_objective(m, G, k, dist_bounds)
            if heuristic:
                heuristic_districts = [ [node  for node in G.nodes  if G.nodes[node]['start_bvap']==j+1] for j in range(k) ]
            else:
                heuristic_districts = None
        elif obj =="conv_approx":
            labeling.add_conv_approx_objective(m, G, k)
            if heuristic:
                heuristic_districts = [ [node  for node in G.nodes  if G.nodes[node]['start_bvap']==j+1] for j in range(k) ]
            else:
                heuristic_districts = None
        elif obj =="PWL_approx":
            labeling.add_PWL_approx_objective(m, G, k, R)
            if heuristic:
                heuristic_districts = [ [node  for node in G.nodes  if G.nodes[node]['start_bvap']==j+1] for j in range(k) ]
            else:
                heuristic_districts = None
        elif obj =="partisan_dem":
            labeling.add_partisan_dem_objective(m, G, k, R,U)
            if heuristic:
                heuristic_districts = [ [node  for node in G.nodes  if G.nodes[node]['start_dem']==j+1] for j in range(k) ]
            else:
                heuristic_districts = None
        elif obj =="partisan_dem_order":
            labeling.add_partisan_dem_objective_ordering(m, G, k, R,U,config['obj_order'],config['index'] )
            if heuristic:
                heuristic_districts = [ [node  for node in G.nodes  if G.nodes[node]['start_dem']==j+1] for j in range(k) ]
            else:
                heuristic_districts = None
        elif obj =="partisan_rep":
            labeling.add_partisan_rep_objective(m, G, k, R,U)
            if heuristic:
                heuristic_districts = [ [node  for node in G.nodes  if G.nodes[node]['start_rep']==j+1] for j in range(k) ]
            else:
                heuristic_districts = None
        elif obj =="competetive":
            labeling.add_comp_objective(m, G, k)
            if heuristic:
                heuristic_districts = [ [node  for node in G.nodes  if G.nodes[node]['start_comp']==j+1] for j in range(k) ]
            else:
                heuristic_districts = None
        elif obj == "step-ordered":
            labeling.add_step_ordered_objective(m, G, k, R,U)
            if heuristic:
                heuristic_districts = [ [node  for node in G.nodes  if G.nodes[node]['start_bvap']==j+1] for j in range(k) ]
            else:
                heuristic_districts = None
        elif obj == "DiscretePWL":
            labeling.add_disctrete_PWL_objective(m, G, k, R)
            if heuristic:
                heuristic_districts = [ [node  for node in G.nodes  if G.nodes[node]['start_bvap']==j+1] for j in range(k) ]
            else:
                heuristic_districts = None
        elif obj == "step-alt":
            (expErr,maxErr) = labeling.add_step_alt_objective(m, G, k, R)
            if heuristic:
                heuristic_districts = [ [node  for node in G.nodes  if G.nodes[node]['start_bvap']==j+1] for j in range(k) ]
            else:
                heuristic_districts = None
        elif obj == "step-hvap":
            labeling.add_step_hvap_objective(m, G, k, R, state)
            if heuristic:
                heuristic_districts = [ [node  for node in G.nodes  if G.nodes[node]['start_bvap']==j+1] for j in range(k) ]
            else:
                heuristic_districts = None
        elif obj == "compact":
            labeling.add_compact_objective2(m, G, k, R, state)
            if heuristic:
                heuristic_districts = [ [node  for node in G.nodes  if G.nodes[node]['start_bvap']==j+1] for j in range(k) ]
            else:
                heuristic_districts = None
        elif obj == "perimax":
            labeling.add_perimax_objective(m, G, k)
            if heuristic:
                heuristic_districts = [ [node  for node in G.nodes  if G.nodes[node]['start_bvap']==j+1] for j in range(k) ]
            else:
                heuristic_districts = None
        elif obj == "perimin":
            labeling.add_perimin_objective(m, G, k)
            if heuristic:
                heuristic_districts = [ [node  for node in G.nodes  if G.nodes[node]['start_bvap']==j+1] for j in range(k) ]
            else:
                heuristic_districts = None
        elif obj == "BNPWL":
            labeling.add_BNPWL_objective(m, G, k, R,U)
            if heuristic:
                heuristic_districts = [ [node  for node in G.nodes  if G.nodes[node]['start_bvap']==j+1] for j in range(k) ]
            else:
                heuristic_districts = None
        elif obj == "LogEPWL":
            labeling.add_LogEPWL_objective(m, G, k, R, U)
            if heuristic:
                heuristic_districts = [ [node  for node in G.nodes  if G.nodes[node]['start_bvap']==j+1] for j in range(k) ]
            else:
                heuristic_districts = None
                
        elif obj == "bounds":
            
            labeling.add_LogEPWL_objective(m, G, k, R)
            if heuristic:
                heuristic_districts = [ [node  for node in G.nodes  if G.nodes[node]['start_bvap']==j+1] for j in range(k) ]
            else:
                heuristic_districts = None
    
    
    
    
    ####################################   
    # Contiguity constraints
    ####################################      
            
    contiguity = config['contiguity']
    m._callback = None
    m._population = population
    m._U = U
    m._k = k
    m._base = base
    m._numLazyCuts = 0
    m._numCallbacks = 0
    
    if base == 'hess':
        if contiguity == 'shir':
            hess.add_shir_constraints(m)
        elif contiguity == 'scf':
            hess.add_scf_constraints(m, G, extended)
        elif contiguity == 'lcut':
            m.Params.lazyConstraints = 1
            m._callback = separation.lcut_separation_generic
                    
    if base == 'labeling':
        if contiguity == 'shir':
            labeling.add_shir_constraints(m, config['symmetry'])
        elif contiguity == 'scf':
            labeling.add_scf_constraints(m, G, extended, config['symmetry'])
        elif contiguity == 'lcut':
            m.Params.lazyConstraints = 1
            m._callback = separation.lcut_separation_generic 
    
    
    m.update()
    
    
    ############################################
    # Vertex ordering and max B problem 
    ############################################  
        
#     order = config['order']
    
#     if order == 'B_decreasing':
#         (B, result['B_q'], result['B_time'], result['B_timelimit']) = ordering.solve_maxB_problem(DG, population, L, k, heuristic_districts)
        
#         # draw set B on map and save
#         fn_B = "../" + "results_for_" + config_filename_wo_extension + "/" + result['state'] + "-" + result['level'] + "-maxB.png"       
#         export_B_to_png(G, df, B, fn_B)
#     else:
#         (B, result['B_q'], result['B_time'], result['B_timelimit']) = (list(),'n/a','n/a', 'n/a')
        
#     result['B_size'] = len(B)
    
#     vertex_ordering = ordering.find_ordering(order, B, DG, population)
#     position = ordering.construct_position(vertex_ordering)
    
#     print("Vertex ordering =", vertex_ordering)  
#     print("Position vector =", position)
#     print("Set B =", B)

    ####################################   
    # Symmetry handling
    ####################################    
    
    symmetry = config['symmetry']
    
    if symmetry == 'aggressive':
        m.Params.symmetry = 2
    elif symmetry == 'orbitope':
        if base == 'labeling':
            labeling.add_orbitope_extended_formulation(m, G, k, vertex_ordering)
        else:
            sys.exit("Error: orbitope only available for labeling base model.")     
    
    m.Params.symmetry = 0
            
    ####################################   
    # Variable fixing
    ####################################    
    
    do_fixing = config['fixing']
    
    if do_fixing and base == 'hess':
        result['DFixings'] = fixing.do_Hess_DFixing(m, G, position)
        result['UFixings_R'] = 'n/a'
        
        if contiguity == 'none':
            result['LFixings'] = fixing.do_Hess_LFixing_without_Contiguity(m, G, population, L, vertex_ordering)
            result['UFixings_X'] = fixing.do_Hess_UFixing_without_Contiguity(m, G, population, U)
        else:
            result['LFixings'] = fixing.do_Hess_LFixing(m, G, population, L, vertex_ordering)
            result['UFixings_X'] = fixing.do_Hess_UFixing(m, DG, population, U, vertex_ordering)         
        
        if extended:
            result['ZFixings'] = fixing.do_Hess_ZFixing(m, G)
        else:
            result['ZFixings'] = 0
                
    
    if do_fixing and base == 'labeling':
        #result['DFixings'] = fixing.do_Labeling_DFixing(m, G, vertex_ordering, k)
        
        if contiguity == 'none':
            if symmetry == 'orbitope':
                1
                #result['LFixings'] = fixing.do_Labeling_LFixing_without_Contiguity(m, G, population, L, vertex_ordering, k)
            else:
                result['LFixings'] = 0
            (result['UFixings_X'], result['UFixings_R']) = fixing.do_labeling_UFixing_without_Contiguity()
        #else:
            #result['LFixings'] = fixing.do_Labeling_LFixing(m, G, population, L, vertex_ordering, k)
            #(result['UFixings_X'], result['UFixings_R']) = fixing.do_Labeling_UFixing(m, DG, population, U, vertex_ordering, k)
        
        if extended:
            result['ZFixings'] = fixing.do_Labeling_ZFixing(m, G, k)
        else:
            result['ZFixings'] = 0
            
    if not do_fixing:
        result['DFixings'] = 0
        result['UFixings_R'] = 0
        result['LFixings'] = 0
        result['UFixings_X'] = 0
        result['ZFixings'] = 0
            
    
    ######################################################################################
    # Solve root LP? Used only for reporting purposes. Not used for MIP solve.
    ######################################################################################  
    
    if config['lp']:
        r = m.relax() # LP relaxation of MIP model m
        r.Params.LogToConsole = 0 # keep log to a minimum
        r.Params.Method = 3 # use concurrent LP solver
        r.Params.TimeLimit = 3600 # one-hour time limit for solving LP
        print("To get the root LP bound, now solving a (separate) LP model.")
        
        lp_start = time.time()
        r.optimize()
        lp_end = time.time()
        
        if r.status == GRB.OPTIMAL:
            result['LP_obj'] = '{0:.2f}'.format(r.objVal)
        elif r.status == GRB.TIME_LIMIT:
            result['LP_obj'] = 'TL'
        else:
            result['LP_obj'] = '?'
        result['LP_time'] = '{0:.2f}'.format(lp_end - lp_start)
        
    else:
        result['LP_obj'] = 'n/a'
        result['LP_time'] = 'n/a'
        
    
    ####################################   
    # Inject heuristic warm start
    ####################################    
    
    if heuristic and base == 'hess':
        for district in heuristic_districts:    
            p = min([position[v] for v in district])
            j = vertex_ordering[p]
            for i in district:
                m._X[i,j].start = 1
               
    print("\n start stuff \n")            
    if heuristic and base == 'labeling':
        print('skip ordered sol add')
        #m.read("ordering test_44_reordered.sol");
        
        # m.update();
#         flag = 0
#         for i in G.nodes:
#             count = 0
#             for nodes in heuristic_districts:
#                 if i in nodes:
#                     count = count+1
#             if count != 1:
#                 print(i, count)
#                 flag = 1
#         if flag:
#             print("Error")
                
            
                
#         for i,nodes in enumerate(heuristic_districts):
#             for node in nodes:
#                 m._X[node,i].start = int(1)
                
#                 for j in range(k):
#                     if i != j:
#                         m._X[node,j].start = 0
        
#         center_positions = [ min( position[v] for v in heuristic_districts[j] ) for j in range(k) ] 
#         cplabel = { center_positions[j] : j for j in range(k) }
    
#         # what node r will root the new district j? The one with earliest position.
#         for j in range(k):
#             min_cp = min(center_positions)
#             r = vertex_ordering[min_cp]
#             old_j = cplabel[min_cp]
            
#             for i in heuristic_districts[old_j]:
#                 m._X[i,j].start = 1
                
#             center_positions.remove(min_cp)
                
    
    ####################################   
    # Solve MIP
    ####################################  
    
    result['MIP_timelimit'] = config['tlimit'] 
    m.Params.TimeLimit = result['MIP_timelimit']
    m.Params.Method = 3 # use concurrent method for root LP. Useful for degenerate models
    
    fn = "../" + "results_for_" + config_filename_wo_extension + "/" + result['state'] + "-" + result['level'] + "-" + config['obj'] + "-" + str(config['R'])
    m.params.LogFile= fn+".log"
    
    
    #zz = m.addVar(vtype = GRB.INTEGER,name = 'z' )
    #zzz = m.addVar(vtype = GRB.INTEGER,name = 'zzz', lb = 0, ub = 4 )
    
    #best_district = [0, 2, 34, 4, 35, 36, 9, 44, 14, 16, 17, 19, 21, 27, 28, 30]
    #m.addConstr(4*zz == gp.quicksum(m._X[i,6] for i in best_district) + zzz)
    #zz.branchPriority = 0
    #next_best = [1, 15, 22, 23, 33, 39]
    #m.addConstr(gp.quicksum(m._X[i,6] for i in best_district)>= 10)
    #m.addConstr(gp.quicksum(m._X[i,5] for i in next_best)>= 4)
    #m.setParam('LogToConsole', 0)
    m.write("this_optimization_model.lp")
    start = time.time()
    m.optimize(m._callback)
    end = time.time()
    result['MIP_time'] = '{0:.2f}'.format(end-start)
    
    result['MIP_status'] = int(m.status)
    result['MIP_nodes'] = int(m.NodeCount)
    result['MIP_bound'] = m.objBound
    result['callbacks'] = m._numCallbacks
    result['lazy_cuts'] = m._numLazyCuts
    if obj == "step-exp" or obj == "step-max":
        result['expected_error'] = expErr
        result['maximum_error'] = maxErr
    
    # report best solution found
    if m.SolCount > 0:
        result['MIP_obj'] = m.objVal

        if base == 'hess':
            labels = [ j for j in DG.nodes if m._X[j,j].x > 0.5 ]
        else: # base == 'labeling'
            labels = [ j for j in range(k) ]
            
        districts = [ [ i for i in DG.nodes if m._X[i,j].x > 0.5 ] for j in labels]
        print("best solution (found) =",districts)
        
        # export solution to .json file
        m.write(fn+'.sol')
        json_fn = fn + ".json"
        export_to_json(G, districts, json_fn)
        
        # export solution to .png file (districting map)
        png_fn = fn + ".png"
        export_to_png(G, df, districts, png_fn)
        
        # is solution connected?
        connected = True
        for district in districts:
            if not nx.is_connected(G.subgraph(district)):
                connected = False
        result['connected'] = connected
        
    else:
        result['MIP_obj'] = 'no_solution_found'
        result['connected'] = 'n/a'
        
            
    ####################################   
    # Summarize results of this run to csv file
    ####################################  
    for name in list(result.keys()):
        if name not in my_fieldnames:
            my_fieldnames.append(name)
    append_dict_as_row(results_filename,result,my_fieldnames)
    

Reading config from config-rob-tests-BNPWL.json
In run AL_step-ordered_90_rob_test_new_loge using config: {'state': 'AL', 'level': 'county', 'base': 'labeling', 'obj': 'LogEPWL', 'R': 90, 'dist_bounds': False, 'tlimit': 100000, 'fixing': False, 'contiguity': 'lcut', 'symmetry': 'default', 'extended': False, 'order': 'B_decreasing', 'heuristic': True, 'lp': False, 'obj_order': 'D20_max', 'index': 1}.
../data/county/dual_graphs/county01.json

L = 645979 , U = 789529 , k = 7
Academic license - for non-commercial use only - expires 2023-09-22
Using license file C:\Users\robert\gurobi.lic
{'state': 'AL', 'level': 'county', 'base': 'labeling', 'obj': 'LogEPWL', 'R': 90, 'dist_bounds': False, 'tlimit': 100000, 'fixing': False, 'contiguity': 'lcut', 'symmetry': 'default', 'extended': False, 'order': 'B_decreasing', 'heuristic': True, 'lp': False, 'obj_order': 'D20_max', 'index': 1, 'run': 'AL_step-ordered_90_rob_test_new_loge', 'k': 7, 'L': 645979, 'U': 789529, 'm': 171, 'n': 67, 'heur_obj': '

 199774 77698    3.50011  116   29          -    3.50011      -  58.1  196s
 204490 78530    3.50011  122   41          -    3.50011      -  58.0  205s
 208601 79107 infeasible  198               -    3.50011      -  58.0  214s
 212871 79761    3.50011   73   96          -    3.50011      -  57.9  219s
 215163 81112    3.50011   57   94          -    3.50011      -  58.1  224s
 217953 82656    3.50011   82   27          -    3.50011      -  58.3  229s
 220417 83921 infeasible   95               -    3.50011      -  58.6  232s
 222373 85198    3.50011  104   62          -    3.50011      -  59.0  236s
 225971 87242 infeasible  140               -    3.50011      -  59.5  242s
 227611 88385    3.50011   85   44          -    3.50011      -  60.0  245s
 231190 90587    3.50011   95   55          -    3.50011      -  60.4  252s
 233216 91971    3.50011  116   26          -    3.50011      -  60.6  260s
 236113 93136    3.50011  110   21          -    3.50011      -  60.8  265s
 238508 9419

 501581 148276    3.50011   91   82          -    3.50011      -  74.5 1177s
 508890 148534    3.50011  115   36          -    3.50011      -  74.0 1208s
 516470 147152 infeasible  115               -    3.50011      -  73.6 1220s
 520574 147067 infeasible   83               -    3.50011      -  73.6 1238s
 526289 146273 infeasible  100               -    3.50011      -  73.5 1251s
 530644 145863    3.50011   75   58          -    3.50011      -  73.4 1260s
 533907 145438 infeasible   81               -    3.50011      -  73.5 1269s
 537648 144688    3.50011  114   60          -    3.50011      -  73.6 1276s
 541212 143930 infeasible  124               -    3.50011      -  73.6 1283s
 543892 143539    3.50011  111   61          -    3.50011      -  73.8 1288s
 545356 143669    3.50011   65   64          -    3.50011      -  74.1 1293s
 547150 143725 infeasible   94               -    3.50011      -  74.2 1298s
 548948 143590    3.50011   84   59          -    3.50011      -  74.5 1303s

 757232 107155 infeasible  105               -    3.50011      -  98.5 1901s
 760336 108137    3.50011  108   73          -    3.50011      -  98.6 1914s
 764856 108223    3.50011  107   76          -    3.50011      -  98.6 1924s
 767975 108317 infeasible  115               -    3.50011      -  98.7 1933s
 771590 108521    3.50011  112   67          -    3.50011      -  98.8 1955s
 776296 108773    3.50011  112   76          -    3.50011      -  98.9 1991s
 781191 108846 infeasible  101               -    3.50011      -  98.9 2000s
 784143 108919    3.50011  103   77          -    3.50011      -  99.3 2007s
 786264 109014    3.50011  124   86          -    3.50011      -  99.5 2019s
 790088 108919 infeasible  101               -    3.50011      -   100 2026s
 792508 109073 infeasible  110               -    3.50011      -   100 2034s
 794966 110443 infeasible   95               -    3.50011      -   100 2062s
 803999 110243 infeasible  155               -    3.50011      -   100 2081s

 1077644 136899    3.50011  109  105          -    3.50011      -   118 3032s
 1080354 137073    3.50011   95  100          -    3.50011      -   118 3039s
 1083083 137298 infeasible  117               -    3.50011      -   119 3051s
 1086649 137539 infeasible  110               -    3.50011      -   119 3062s
 1089798 137708    3.50011   87  105          -    3.50011      -   119 3070s
 1092255 137940    3.50011   95   57          -    3.50011      -   119 3079s
 1095406 138008 infeasible  106               -    3.50011      -   119 3089s
 1098463 138176    3.50011  116   72          -    3.50011      -   119 3098s
 1101664 138369 infeasible   97               -    3.50011      -   119 3106s
 1103350 138555    3.50011   84   58          -    3.50011      -   119 3115s
 1105842 138762 infeasible  107               -    3.50011      -   120 3124s
 1108762 138989 infeasible   97               -    3.50011      -   120 3132s
 1111345 139315    3.50011  103  101          -    3.50011      

...

 7394958 362031    3.50011   78   80          -    3.50011      -   212 44926s
 7397906 362476    3.50011   79  120          -    3.50011      -   212 44959s
 7401505 362313 infeasible   91               -    3.50011      -   212 44984s
 7404490 362430 infeasible   84               -    3.50011      -   212 45006s
 7407102 362653 infeasible   74               -    3.50011      -   212 45024s
 7409026 362711    3.50011   77   92          -    3.50011      -   212 45045s
 7411360 362802 infeasible   90               -    3.50011      -   212 45071s
 7414318 362887 infeasible   98               -    3.50011      -   212 45095s
 7416913 362937    3.50011   65  107          -    3.50011      -   212 45128s
 7420500 363011 infeasible   97               -    3.50011      -   212 45158s
 7423689 363142 infeasible  102               -    3.50011      -   212 45194s
 7427344 363139 infeasible   96               -    3.50011      -   212 45216s
 7429642 363170 infeasible   85               -    3

 7708772 367267 infeasible   85               -    3.50011      -   214 48021s
 7710348 367277 infeasible   98               -    3.50011      -   214 48044s
 7711031 367277 infeasible   95               -    3.50011      -   214 48045s
 7712772 367308 infeasible   92               -    3.50011      -   214 48074s
 7715333 367334    3.50011   96   86          -    3.50011      -   214 48100s
 7717900 367400 infeasible  108               -    3.50011      -   214 48124s
 7720693 367372 infeasible   95               -    3.50011      -   214 48153s
 7723319 367477 infeasible  104               -    3.50011      -   214 48192s
 7726914 367570    3.50011   93   80          -    3.50011      -   214 48214s
 7729372 367692    3.50011   77  102          -    3.50011      -   214 48241s
 7732211 367588    3.50011   85   98          -    3.50011      -   214 48266s
 7734862 367672    3.50011   81   96          -    3.50011      -   214 48313s
 7739530 367764    3.50011   91   79          -    3

 7981671 376440    3.50011   95  112          -    3.50011      -   216 50866s
 7983908 376391 infeasible   86               -    3.50011      -   216 50908s
 7987244 376396    3.50011   89   79          -    3.50011      -   216 50939s
 7990632 376329    3.50011   80   95          -    3.50011      -   216 50973s
 7993657 376393 infeasible   87               -    3.50011      -   216 51014s
 7997341 376428    3.50011   85  107          -    3.50011      -   216 51037s
 7999774 376410    3.50011   96   94          -    3.50011      -   216 51061s
 8002415 376409    3.50011   81  107          -    3.50011      -   216 51094s
 8005683 376450 infeasible  102               -    3.50011      -   216 51130s
 8008912 376508 infeasible   90               -    3.50011      -   216 51154s
 8011189 376625 infeasible   87               -    3.50011      -   216 51184s
 8014007 376632    3.50011   85  117          -    3.50011      -   216 51204s
 8016178 376753    3.50011   88   89          -    3

 8288011 392048 infeasible   69               -    3.50011      -   217 54165s
 8288788 392142 infeasible   85               -    3.50011      -   217 54207s
 8292971 392427 infeasible   74               -    3.50011      -   217 54240s
 8295367 392610    3.50011   85  108          -    3.50011      -   217 54264s
 8295845 392610    3.50011   95  111          -    3.50011      -   217 54265s
 8297774 392761    3.50011  105  108          -    3.50011      -   217 54290s
 8300153 392897 infeasible   90               -    3.50011      -   217 54314s
 8302594 392975 infeasible  100               -    3.50011      -   217 54339s
 8303476 392975    3.50011   90  100          -    3.50011      -   217 54340s
 8304976 393134    3.50011   89  109          -    3.50011      -   217 54372s
 8307481 393252    3.50011   94  119          -    3.50011      -   217 54398s
 8309953 393330 infeasible  106               -    3.50011      -   217 54432s
 8312531 393483    3.50011  102  110          -    3

 8622740 408409 infeasible   70               -    3.50011      -   217 58044s
 8625437 408510    3.50011   86  116          -    3.50011      -   217 58063s
 8627174 408583 infeasible   79               -    3.50011      -   217 58098s
 8630766 408654    3.50011   60   79          -    3.50011      -   217 58126s
 8633376 408788    3.50011   65   76          -    3.50011      -   217 58150s
 8635809 408981    3.50011   76   87          -    3.50011      -   217 58182s
 8638735 409186 infeasible   97               -    3.50011      -   217 58212s
 8641454 409262 infeasible   66               -    3.50011      -   217 58241s
 8644261 409416    3.50011   85  100          -    3.50011      -   217 58271s
 8646873 409564    3.50011   89   80          -    3.50011      -   217 58301s
 8649436 409592 infeasible  104               -    3.50011      -   217 58332s
 8652241 409766    3.50011   95  106          -    3.50011      -   217 58358s
 8654812 409838 infeasible   94               -    3

In [6]:
result

{'state': 'SC',
 'level': 'county',
 'base': 'labeling',
 'obj': 'partisan_dem_order',
 'R': 90,
 'dist_bounds': False,
 'tlimit': 60,
 'fixing': False,
 'contiguity': 'lcut',
 'symmetry': 'default',
 'extended': False,
 'order': 'B_decreasing',
 'heuristic': True,
 'lp': False,
 'obj_order': 'D20_max',
 'index': 6,
 'run': 'SC_dem_bounds6',
 'k': 7,
 'L': 658084,
 'U': 804323,
 'm': 109,
 'n': 46,
 'heur_obj': 'n/a',
 'heur_time': 'n/a',
 'heur_iter': 'n/a',
 'DFixings': 0,
 'UFixings_R': 0,
 'LFixings': 0,
 'UFixings_X': 0,
 'ZFixings': 0,
 'LP_obj': 'n/a',
 'LP_time': 'n/a',
 'MIP_timelimit': 60,
 'MIP_time': '60.21',
 'MIP_status': 9,
 'MIP_nodes': 99925,
 'MIP_bound': 246587.75947995664,
 'callbacks': 85,
 'lazy_cuts': 984,
 'MIP_obj': 'no_solution_found',
 'connected': 'n/a'}

In [10]:
my_fieldnames == list(result.keys())

False

In [11]:
for name in my_fieldnames:
    if name not in list(result.keys()):
        print(name)

B_q
B_size
B_time
B_timelimit
expected_error
maximum_error


In [12]:
for name in list(result.keys()):
    if name not in my_fieldnames:
        print(name)

obj_order
index


In [ ]:
Added ordering D20[j]   --> Slightly better, but still worse than original run.
    0     0    6.97621    0  915          -    6.97621      -     -   53s
     0     0    6.97600    0  913          -    6.97600      -     -   58s
     0     0          -    0               -    6.97600      -     -   60s


In [ ]:
  Added D[j] <= T[j] ---> Did worse!
    0     0    6.97717    0  948    1.88990    6.97717   269%     -   46s
     0     0    6.97690    0  932    1.88990    6.97690   269%     -   50s
     0     0    6.97618    0  970    1.88990    6.97618   269%     -   56s
     0     0          -    0         1.88990    6.97618   269%     -   60s

In [5]:

Original Run 60 seconds (Dem objective SC)
0     0    6.97717    0  948    1.88990    6.97717   269%     -   46s
     0     0    6.97690    0  932    1.88990    6.97690   269%     -   50s
     0     0    6.97618    0  970    1.88990    6.97618   269%     -   56s
     0     0    6.97563    0  967    1.88990    6.97563   269%     -   59s

SyntaxError: invalid syntax (<ipython-input-5-6830bdf6b343>, line 1)

In [3]:
{0: {0: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]
  , 1: [65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89]}, 
 1: {0: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32], 1: [33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89]}, 
 2: {0: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80], 
     1: [17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 81, 82, 83, 84, 85, 86, 87, 88, 89]}, 
 3: {0: [1, 2, 3, 4, 5, 6, 7, 8, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 89], 
     1: [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88]}, 
 4: {0: [1, 2, 3, 4, 13, 14, 15, 16, 17, 18, 19, 20, 29, 30, 31, 32, 33, 34, 35, 36, 45, 46, 47, 48, 49, 50, 51, 52, 61, 62, 63, 64, 65, 66, 67, 68, 77, 78, 79, 80, 81, 82, 83, 84], 
     1: [5, 6, 7, 8, 9, 10, 11, 12, 21, 22, 23, 24, 25, 26, 27, 28, 37, 38, 39, 40, 41, 42, 43, 44, 53, 54, 55, 56, 57, 58, 59, 60, 69, 70, 71, 72, 73, 74, 75, 76, 85, 86, 87, 88, 89]}, 
 5: {0: [1, 2, 7, 8, 9, 10, 15, 16, 17, 18, 23, 24, 25, 26, 31, 32, 33, 34, 39, 40, 41, 42, 47, 48, 49, 50, 55, 56, 57, 58, 63, 64, 65, 66, 71, 72, 73, 74, 79, 80, 81, 82, 87, 88, 89], 
     1: [3, 4, 5, 6, 11, 12, 13, 14, 19, 20, 21, 22, 27, 28, 29, 30, 35, 36, 37, 38, 43, 44, 45, 46, 51, 52, 53, 54, 59, 60, 61, 62, 67, 68, 69, 70, 75, 76, 77, 78, 83, 84, 85, 86]}, 
 6: {0: [1, 4, 5, 8, 9, 12, 13, 16, 17, 20, 21, 24, 25, 28, 29, 32, 33, 36, 37, 40, 41, 44, 45, 48, 49, 52, 53, 56, 57, 60, 61, 64, 65, 68, 69, 72, 73, 76, 77, 80, 81, 84, 85, 88, 89], 
     1: [2, 3, 6, 7, 10, 11, 14, 15, 18, 19, 22, 23, 26, 27, 30, 31, 34, 35, 38, 39, 42, 43, 46, 47, 50, 51, 54, 55, 58, 59, 62, 63, 66, 67, 70, 71, 74, 75, 78, 79, 82, 83, 86, 87]}}


Changed value of parameter LogFile to mylog.log
   Prev: ../results_for_config-rob-tests-BNPWL/SC-county-LogEPWL-90.log  Default: 


GurobiError: Unknown file type for file 'mylog.log'

In [5]:
### Some value values for SC
vap_lb = [501419,501419,501419,501419,501419,501419,503655]
vap_ub = [655035,655035,655035,655035,655035,655035, 653780]

bvap_ub = [137826, 149052, 163457, 185065, 216833, 254706, 304582]
bvap_lb = [77028, 77028, 77028, 77028, 95428, 111127, 172777]



In [ ]:
b

In [ ]:
-271530.72

In [7]:
math.sqrt(bvap_ub[6]**2 + vap_ub[6]**2)

721247.8652474474

In [2]:
math.sqrt(2.835202507540e+11)

532466.1968181642

In [8]:
math.sqrt(bvap_lb[6]**2 + vap_lb[6]**2)

532466.1968181642

In [3]:
math.sqrt(4.962661333040e+11)

704461.5910778955

In [ ]:
 263533 163809    1.34901   63   46    1.33573    1.46075  9.36%  16.6   50s
 280839 173326    1.33637   53   40    1.33573    1.45941  9.26%  17.0   55s
 297310 181677    1.43720   43   70    1.33573    1.45761  9.12%  17.6   60s

In [22]:
fn

'../results_for_config-rob-tests-BNPWL/SC-county-LogEPWL-12'

In [10]:
q = m.getVarByName('f')

In [13]:
lam[6,7,in] 0
lam[6,7,out] 4.2334955521418255e-01
lam[6,8,in] 5.5573493325975332e-01
lam[6,8,out] 2.0915511526064023e-02

In [26]:
l7out= m.getVarByName(f'lam[6,7,out]').x

In [27]:
l8in = m.getVarByName(f'lam[6,8,in]').x

In [28]:
l8out = m.getVarByName(f'lam[6,8,out]').x

In [29]:
l7out + l8in + l8out

0.9999999999999998

In [15]:
for j in range(k):
    print(m.getVarByName(f'f[{j}]'))

<gurobi.Var f[0] (value 0.039799453702341364)>
<gurobi.Var f[1] (value 0.07359627072208957)>
<gurobi.Var f[2] (value 0.07616226720690616)>
<gurobi.Var f[3] (value 0.07884219387784674)>
<gurobi.Var f[4] (value 0.08653958110047141)>
<gurobi.Var f[5] (value 0.525263346654526)>
<gurobi.Var f[6] (value 0.6105529590708061)>


In [16]:
for j in range(k):
    print(m.getVarByName(f'y[{j}]'))

<gurobi.Var y[0] (value 629679.0)>
<gurobi.Var y[1] (value 537662.0)>
<gurobi.Var y[2] (value 595591.0)>
<gurobi.Var y[3] (value 583282.0)>
<gurobi.Var y[4] (value 631354.0)>
<gurobi.Var y[5] (value 517860.0)>
<gurobi.Var y[6] (value 519040.0)>


In [17]:
for j in range(k):
    print(m.getVarByName(f'z[{j}]'))

<gurobi.Var z[0] (value 89262.0)>
<gurobi.Var z[1] (value 92355.0)>
<gurobi.Var z[2] (value 108679.0)>
<gurobi.Var z[3] (value 109767.0)>
<gurobi.Var z[4] (value 131180.0)>
<gurobi.Var z[5] (value 217520.0)>
<gurobi.Var z[6] (value 222581.0)>


In [18]:
for j in range(k):
    print(m.getVarByName(f'z[{j}]').x/m.getVarByName(f'y[{j}]').x)

0.1417579433330316
0.17177148468740583
0.18247253568304422
0.18818856059333222
0.2077756694342635
0.4200363032479821
0.42883207459926015


In [21]:
for j in range(k):
    print([labeling.cdf_fun(m.getVarByName(f'z[{j}]').x/m.getVarByName(f'y[{j}]').x), m.getVarByName(f'f[{j}]').x])

[0.03148804404064194, 0.039799453702341364]
[0.04901421544200262, 0.07359627072208957]
[0.05688845146909919, 0.07616226720690616]
[0.06148518907470151, 0.07884219387784674]
[0.07945850412978173, 0.08653958110047141]
[0.5160203279901475, 0.525263346654526]
[0.5399102998242208, 0.6105529590708061]


In [ ]:
sum((lam[d,i,'in']+lam[d,i,'out'])*f_bvap(B[i][1]/B[i][0]) for i in range(L)) for d in D)

In [ ]:
51810 27489    1.53642   38  125    1.36246    1.74488  28.1%   135   56s
 57410 30386    1.54474   39  118    1.36246    1.73821  27.6%   135   60s


In [ ]:
inf and sup ratios: 0.06407776152503185, 0.6846119336025124

In [1]:
304582/501419

0.6074400850386603

In [2]:
77028/653780

0.1178194499678791

In [ ]:
    ## population variables
    vap_lb = [501419,501419,501419,501419,501419,501419,503655]
    vap_ub = [655035,655035,655035,655035,655035,655035, 653780]
    
    vap = {j : m.addVar(name = f"vap{j}", ub = vap_ub[j], lb = vap_lb[j])   for j in range(k)} # voting age population in district j
    
    bvap_ub = [137826, 149052, 163457, 185065, 216833, 254706, 304582]
    bvap_lb = [77028, 77028, 77028, 77028, 95428, 111127, 172777]
    bvap = {j : m.addVar(name = f"bvap{j}", ub = bvap_ub[j], lb = 77028)  for j in range(k)} # bvap in district j

In [2]:
501419 + 172777

674196

In [ ]:
precomputed bounds for each order + bvap ordering
 129822 64495 infeasible   78               -    1.70667      -  71.7   55s
 160374 79927    1.54167   53  112          -    1.69631      -  72.0   60s (slight improvement...)

In [ ]:
precomputed bounds + bvap ordering
 56906 24123    1.57094   56  114          -    1.79019      -  72.6   45s
 84444 34548 infeasible  102               -    1.74728      -  70.7   50s
 115157 50956    1.28257   58   73          -    1.71752      -  67.0   56s
 139810 62258 infeasible   76               -    1.70287      -  61.9   60s (Way better!)

In [ ]:
Precomputed bounds + objective ordering
 2920  2485    2.17217   18  489    1.37979    2.17217  57.4%   111   50s
  2976  2540    2.15457   45  449    1.37979    2.17005  57.3%   116   55s
  3873  2841    1.46347   61  133    1.37979    2.17005  57.3%   154   60s (Slower!)

In [ ]:
Adding precomputed upper and lower bounds on vap and bvap (no objective ordering)
 32558 15733    1.99501   65  237    1.37979    2.08772  51.3%  74.0   50s
 45220 22389    1.55480   89   56    1.37979    2.06289  49.5%  75.4   55s
 58924 28984    1.79221   70  139    1.37979    2.04846  48.5%  76.0   60s

In [ ]:
Adding just upper bounds on Vap and also Vap >= bvap
  2722  2255    1.72433   61  606    1.37979    2.45934  78.2%  47.1   50s
  2738  2266    2.45765   25  602    1.37979    2.45765  78.1%  46.9   55s

In [ ]:
fancy delta inequality
2765  2398    2.38401   40  598    1.37979    2.52117  82.7%  86.8   50s
  2781  2408    2.49282   13  601    1.37979    2.51811  82.5%  86.3   55s

In [ ]:
no ordering
3347  2738    2.50354   21  591    1.37979    2.50354  81.4%  52.0   45s
  3363  2748    2.50061   22  585    1.37979    2.50061  81.2%  51.8   50s
  3378  2758    2.49687   35  590    1.37979    2.49687  81.0%  51.6   55s

In [ ]:
ordered cdf, but not deltas
  2765  2253    1.61491   74  575    1.37979    2.47024  79.0%  67.9   50s
  2785  2266    2.46641    8  588    1.37979    2.46641  78.8%  67.4   55s

In [ ]:
  ordered deltas
    2702  2244    2.52746   11  591    1.37979    2.52746  83.2%   100   50s
  2715  2253    1.49713   73  580    1.37979    2.52575  83.1%  99.4   55s

In [ ]:
2749  2242    2.07990   56  560    1.37979    2.46981  79.0%  68.3   45s
  2768  2255    2.46172   15  583    1.37979    2.46172  78.4%  67.8   50s
  2789  2269    2.45790   17  590    1.37979    2.45790  78.1%  67.3   55s

In [ ]:
  2653  2142    2.50043   15  583    1.37979    2.50043  81.2%  43.8   40s
  2672  2156    2.49879   12  488    1.37979    2.49879  81.1%  66.1   45s
  2692  2169    2.48653   42  586    1.37979    2.48653  80.2%  65.6   50s
  2708  2180    2.33831   16  606    1.37979    2.48365  80.0%  65.2   55s

In [ ]:
for j in range(k):
    print(m.getVarByName(f'cdf{j}'))

In [ ]:
for j in range(k):
    print(j, m.getVarByName(f'bvap{j}').x)

In [1]:
def plot_bvap(dissolved,filename, additional_file_name = '', overlay = False, gdf2 = 0):
        dissolved['Representative'] = dissolved['geometry'].apply(lambda x: x.representative_point().coords[:])
        dissolved['Representative'] = [coords[0] for coords in dissolved['Representative']]
        dissolved = dissolved.rename(columns={'P0010001':'POP','P0010004':'BPOP',
                                                  'P0030001':'VAP','P0030004':'BVAP', 'P0040002' : 'HVAP'})
        dissolved['BVAP_ratio'] = dissolved['BVAP']/dissolved['VAP']
        colormap ='BuPu'
        
        fig, ax = plt.subplots(1,1, figsize=(16,16))
        
        cbax = fig.add_axes([0.95, 0.3, 0.03, 0.39])   
        cbax.set_title("BVAP Ratio")
        vmin = 0
        vmax = 0.75
        sm = plt.cm.ScalarMappable(cmap=colormap, \
                norm=plt.Normalize(vmin=vmin, vmax=vmax))
    
        sm._A = []
    
        fig.colorbar(sm, cax=cbax)#, format="%d")
    
        ax.axis("off")

        dissolved.plot(column = 'BVAP_ratio', edgecolor='black', cmap=colormap, ax = ax, vmin = vmin, vmax = vmax, legend=False)
        #dissolved.apply(lambda x: 
             #ax.annotate(text=x.NAME, xy=x.Representative, ha='center', color = 'r',  weight='bold', size = 14), axis=1);
        
        #dissolved.set_index('my_index', inplace=True)
        dissolved.reset_index(inplace=True)
        #dissolved.apply(lambda x: 
             #ax.annotate(text=str(x.name) +': '+ str(x.BVAP), xy=x.Representative, ha='center', color = 'r',  weight='bold', size = 14), axis=1);
        
        dissolved.apply(lambda x: 
             ax.annotate(text=str(x.name) +': '+ str(round(labeling.cdf_fun(x.BVAP_ratio),2)), xy=x.Representative, ha='center', color = 'r',  weight='bold', size = 14), axis=1);
        
        if overlay:
            gdf2.plot(ax=ax, edgecolor='blue', linewidth=2, facecolor='none')
        plt.savefig(filename.split(".")[0] + '_BVAP' + additional_file_name + '.png', transparent=True)


In [ ]:
labeling.cdf_fun(int(G.nodes[42]['BVAP'])/int(G.nodes[42]['VAP']))

In [5]:
for j in range(k):
    print(m.getVarByName(f'bvap{j}').x, m.getVarByName(f'vap{j}').x, m.getVarByName(f'bvap{j}').x/m.getVarByName(f'vap{j}').x)

89262.0 629679.0 0.1417579433330316
92355.0 537662.0 0.17177148468740583
108679.0 595591.0 0.18247253568304422
109767.0 583282.0 0.18818856059333222
131180.0 631354.0 0.2077756694342635
217520.0 517860.0 0.4200363032479821
222581.0 519040.0 0.42883207459926015


In [ ]:
    vap_lb = [501419,501419,501419,501419,501419,501419,503655]
    vap_ub = [655035,655035,655035,655035,655035,655035, 653780]
    
    bvap_ub = [137826, 149052, 163457, 185065, 216833, 254706, 304582]
    bvap_lb = [77028, 77028, 77028, 77028, 95428, 111127, 172777]

In [ ]:
for j in range(k):
    print(m.getVarByName(f'cdf{j}').x)

In [ ]:
 2585  2155    1.69849   65  484    1.37979    2.79425   103%  36.8   15s
  2617  2176    2.77176   24  500    1.37979    2.77176   101%  36.3   20s
  2643  2193    2.76157   34  531    1.37979    2.76157   100%  36.0   25s

In [ ]:
dissolved

In [ ]:
df

In [ ]:
G.nodes[0]['VAP']

In [ ]:
df['P0030004'][0] df['P0030001'][0]

In [ ]:
df['P0030001'][0]